In [1]:
import os                          # Import operating system interface
import win32com.client as win32    # Import COM
import numpy as np
import matplotlib.pyplot as plt

# Abriendo archivo de Aspen Con Python

In [2]:
# Si esta en la misma carptea solo se marca el nombre del archivo formato bkp.
file = 'S_7.bkp'
# Dirección del archivo
aspen_Path = os.path.abspath(file)

# Conexión a Aspen ...

In [3]:
# Se inicia aplicación de aspen
print('\n Connecting to the Aspen Plus... Please wait ')
Application = win32.Dispatch('Apwn.Document') # Registered name of Aspen Plus
print('Connected!')


 Connecting to the Aspen Plus... Please wait 
Connected!


# Se inicia archivo de Aspen y se oculta el archivo 
No se abre aspen para agilizar el proceso :)

In [4]:
# Se inicia el archivo ASPEN
Application.InitFromArchive2(aspen_Path)

# Se hace el archivo no visible
Application.visible = 0

# Optimización con pymoo
Se emplea el algoritmo NSGA, para mayor información ver referencias. 
Se puede implementar otro metodo como el Particle Swap Optimization.
Este caso solo minimiza una función objetivo (TAC) pero se pueden formular problemas multiobjetivos, como: sustentabilidad, indicador de riesgo, etc.


https://pymoo.org/algorithms/moo/nsga2.html

https://pymoo.org/algorithms/soo/pso.html

En el código vienen algunos comentarios para entenderlo.
Si tienes alguna duda con el código o mejora potencial, Hazmelo saber ¡

In [5]:
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize
from pymoo.core.problem import Problem 
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
import numpy as np

class ColumnC1(Problem):
    
    def __init__(self):
        #Lower y upper bounds de las variables o mejor dicho los rangos de busqueda para las variables 
        #              Netapas,Reflujo,Fstage1,Fstage2,SolventFlow
        xlval = np.array([20,0.5,2 ,2 ,50])
        xuval = np.array([50,1.5,19,19,150])
        #Se definen el número de vairables, funciones objetivos, constraints y se cargar los bounds 
        #de las variables.
        super().__init__(n_var=5, n_obj=1, n_ieq_constr=1, xl=xlval, xu=xuval)

    def _evaluate(self, x, out, *args, **kwargs):
        N_C1 = r"\Data\Blocks\B1\Input\NSTAGE"
        RR_C1  = r"\Data\Blocks\B1\Input\BASIS_RR"
        FeedStage_1  = r"\Data\Blocks\B1\Input\FEED_STAGE\SOLV"  
        FeedStage_2 = r"\Data\Blocks\B1\Input\FEED_STAGE\AZEO"
        solvent_flow_rate = r"\Data\Streams\SOLV\Input\TOTFLOW\MIXED"
        acetone_mole_frac = r"\Data\Streams\ACETO\Output\MOLEFRAC\MIXED\ACETO-01"
        r1 , c1  = x.shape  # Ajuste al population size
        TAC = np.zeros((r1, 1))
        XD1 = np.zeros((r1, 1))
        for i in range(r1):
            Application.Tree.FindNode(N_C1).Value = round(x[i][0])  # se sustituye el valor de N etapas
            Application.Tree.FindNode(RR_C1).Value = x[i][1]  # se sustituye el valor de reflujo
            Application.Tree.FindNode(FeedStage_1).Value = round(x[i][2])  # valor de etapa de alimentacion
            Application.Tree.FindNode(FeedStage_2).Value = round(x[i][3])  # valor de etapa de alimentacion
            Application.Tree.FindNode(solvent_flow_rate).Value = x[i][4]  # se sustituye el valor de Solvent
            Application.Engine.Run2()
            # Outputs
            XD1[i]  = Application.Tree.FindNode(acetone_mole_frac).Value
            COND_DUTY = r"\Data\Blocks\B1\Output\COND_DUTY"
            TOP_TEMP = r"\Data\Blocks\B1\Output\TOP_TEMP"
            BOTTOM_TEMP = r"\Data\Blocks\B1\Output\BOTTOM_TEMP"
            REB_DUTY = r"\Data\Blocks\B1\Output\REB_DUTY"
            DiameterColumnB1 = 1.064321158
            Fm = 1
            Fc = Fm*(1 + 0.0074*(1.0132 - 3.48)  +  0.00023*(1.0132 - 3.48)**2)
            H  = 0.6*(x[i][0]-1) + 2
            ColumnaC1 =  (1716.2/280)*(957.9*(DiameterColumnB1**1.066)*(H**0.802))*(2.18*Fc) 
            PlatosC1 =  (1716.2/280)*(x[i][0])*97.2*(DiameterColumnB1**1.55)
            CapitalCostColumnB1 = ColumnaC1 + PlatosC1
                ## CONDENSADOR B1
            Heattransfercoefficient_KC = 0.852 # kW/ K *m2 
            TemperatureCoolingService = 35 + 273.15 #Kelvyn / Aspen Default 
            HeatCondenser = Application.Tree.FindNode(COND_DUTY).Value*0.0041868  #cal/sec to kW
            T_outputCondenser  = Application.Tree.FindNode(TOP_TEMP).Value + 273.15 #Kelvyn
            DeltaT_Condenser = T_outputCondenser - TemperatureCoolingService
            AreaCondenser = -HeatCondenser/(Heattransfercoefficient_KC*(DeltaT_Condenser)) #m2
            CapitalCostCondenserB1 = 7296*(AreaCondenser**0.65)
                ## REBOILER  B1
            Heattransfercoefficient_KR = 0.568 # kW/ K *m2 
            T_outputReboiler  = Application.Tree.FindNode(BOTTOM_TEMP).Value + 273.15 #Kelvyn
            Tvapor = 164 + 273.15 # Kelvyn Vapor of 100 Psi
            HeatReboiler = Application.Tree.FindNode(REB_DUTY).Value*0.0041868    #cal/sec to kW
            DeltaT_Reboiler = Tvapor - T_outputReboiler
            AreaReboiler = HeatReboiler/(Heattransfercoefficient_KR *DeltaT_Reboiler )
            CapitalCostReboilerB1 =  7296*(AreaReboiler**0.65)
            TotalCapitalCost  = CapitalCostCondenserB1 +  CapitalCostColumnB1 + CapitalCostReboilerB1
            #print('TotalCapitalCost',TotalCapitalCost)
            TAC[i] = (TotalCapitalCost/3)/1e6 
        # out["F"] son las funciones objetivos a evaluar
        # out["G"] son las restricciones del problema, en este caso solo se fija la pureza del destilado
        # se puede poner una senguna restricción para delimitar el rango de concentracion
        # Ejemplo [- XD1 + 0.99980, XD1 - 0.99982]  
        # Aqui el rango entonces queda entre 0.99982 y 0.99980 :) 
        out["F"] = [TAC]
        out["G"] = [- XD1 + 0.9998]

# Se carga problema
problem = ColumnC1()
# Se define el número de generaciones del problema  
termination = get_termination("n_gen", 80)
# Se carga todos los parametros al algoritmo NSGA2
algorithm = NSGA2(pop_size=40,n_offsprings=5,sampling=FloatRandomSampling(),crossover=SBX(prob=0.9, eta=15),mutation=PM(eta=20),eliminate_duplicates=True)
res = minimize(problem,algorithm,termination,seed=1,save_history=True,verbose=True)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       40 |      1 |  0.000000E+00 |  0.2535436907 |             - |             -
     2 |       45 |      1 |  0.000000E+00 |  0.2247110853 |  0.000000E+00 |             f
     3 |       50 |      1 |  0.000000E+00 |  0.1850770426 |  0.000000E+00 |             f
     4 |       55 |      1 |  0.000000E+00 |  0.1365137766 |  0.000000E+00 |             f
     5 |       60 |      1 |  0.000000E+00 |  0.0889079195 |  0.000000E+00 |             f
     6 |       65 |      1 |  0.000000E+00 |  0.0592445007 |  0.000000E+00 |             f
     7 |       70 |      1 |  0.000000E+00 |  0.0399261207 |  0.000000E+00 |             f
     8 |       75 |      1 |  0.000000E+00 |  0.0283356463 |  0.000000E+00 |             f
     9 |       80 |      1 |  0.000000E+00 |  0.0160471360 |  0.000000E+00 |             f
    10 |       85 |      1 |  0.000000E+00 |  0.0070710887 |  0.000000E+00 |             f

In [6]:
print('resultados X',res.X)
print('Function final value',res.F)
print('Constrainst final value ',res.G)
X = res.X
N_C1 = r"\Data\Blocks\B1\Input\NSTAGE"
RR_C1  = r"\Data\Blocks\B1\Input\BASIS_RR"
FeedStage_1  = r"\Data\Blocks\B1\Input\FEED_STAGE\SOLV"  
FeedStage_2 = r"\Data\Blocks\B1\Input\FEED_STAGE\AZEO"
solvent_flow_rate = r"\Data\Streams\SOLV\Input\TOTFLOW\MIXED"
acetone_mole_frac = r"\Data\Streams\ACETO\Output\MOLEFRAC\MIXED\ACETO-01"
Application.Tree.FindNode(N_C1).Value = round(X[0])  # se sustituye el valor de N etapas
Application.Tree.FindNode(RR_C1).Value =X[1] # se sustituye el valor de reflujo
Application.Tree.FindNode(FeedStage_1).Value = round(X[2])  # valor de etapa de alimentacion
Application.Tree.FindNode(FeedStage_2).Value = round(X[3])  # valor de etapa de alimentacion
Application.Tree.FindNode(solvent_flow_rate).Value = X[4]  # se sustituye el valor de flujo de solvente
Application.Engine.Run2()
XD1  = Application.Tree.FindNode(acetone_mole_frac).Value
print('Composicion final de acetona calculado: ', XD1)

resultados X [ 25.88176453   1.06245909   3.10263578  12.64707133 124.06497276]
Function final value [0.10158716]
Constrainst final value  [-5.4415e-05]
Composicion final de acetona calculado:  0.999854493


Nota: Con estos datos , se puede volver a reiterar el problema actualizando los bounds de busqueda para las variables.

El valor de TAC calculado haciendo analisis de sensibilidad y minimizando energia en las columnas se encuentran:    
N Etapas = 29                               
Reflujo  = 1.25                             
Etapa alimentación solvente  = 3               
Etapa alimentación azeotropica = 13         
Flujo de solvente = 101                     
TAC = 0.101458                               
Purezea Destilado | 0.99980               

Con Algoritmo ...

N Etapas Algoritmo = 26     
Reflujo Algoritmo = 1.06    
Etapa alimentación solvente  = 3    (valor redondeado)      
Etapa alimentación azeotropica = 13 (valor redondeado)      
Flujo de solvente = 124.06      
TAC = 0.101587        
Purezea Destilado = 0.99985    


In [9]:
N_C1 = r"\Data\Blocks\B1\Input\NSTAGE"
RR_C1  = r"\Data\Blocks\B1\Input\BASIS_RR"
FeedStage_1  = r"\Data\Blocks\B1\Input\FEED_STAGE\SOLV"  
FeedStage_2 = r"\Data\Blocks\B1\Input\FEED_STAGE\AZEO"
solvent_flow_rate = r"\Data\Streams\SOLV\Input\TOTFLOW\MIXED"
acetone_mole_frac = r"\Data\Streams\ACETO\Output\MOLEFRAC\MIXED\ACETO-01"
Application.Tree.FindNode(N_C1).Value = 29  # se sustituye el valor de N etapas
Application.Tree.FindNode(RR_C1).Value =1.25 # se sustituye el valor de reflujo
Application.Tree.FindNode(FeedStage_1).Value = 3  # valor de etapa de alimentacion
Application.Tree.FindNode(FeedStage_2).Value = 13 # valor de etapa de alimentacion
Application.Tree.FindNode(solvent_flow_rate).Value = 101  # se sustituye el valor de N etapas
Application.Engine.Run2()
XD1  = Application.Tree.FindNode(acetone_mole_frac).Value
print('Composicion final de acetona calculado: ', XD1)
COND_DUTY = r"\Data\Blocks\B1\Output\COND_DUTY"
TOP_TEMP = r"\Data\Blocks\B1\Output\TOP_TEMP"
BOTTOM_TEMP = r"\Data\Blocks\B1\Output\BOTTOM_TEMP"
REB_DUTY = r"\Data\Blocks\B1\Output\REB_DUTY"
DiameterColumnB1 = 1.064321158
Fm = 1
Fc = Fm*(1 + 0.0074*(1.0132 - 3.48)  +  0.00023*(1.0132 - 3.48)**2)
H  = 0.6*(29-1) + 2
ColumnaC1 =  (1716.2/280)*(957.9*(DiameterColumnB1**1.066)*(H**0.802))*(2.18*Fc) 
PlatosC1 =  (1716.2/280)*(29)*97.2*(DiameterColumnB1**1.55)
CapitalCostColumnB1 = ColumnaC1 + PlatosC1
                ## CONDENSADOR B1
Heattransfercoefficient_KC = 0.852 # kW/ K *m2 
TemperatureCoolingService = 35 + 273.15 #Kelvyn / Aspen Default 
HeatCondenser = Application.Tree.FindNode(COND_DUTY).Value*0.0041868  #cal/sec to kW
T_outputCondenser  = Application.Tree.FindNode(TOP_TEMP).Value + 273.15 #Kelvyn
DeltaT_Condenser = T_outputCondenser - TemperatureCoolingService
AreaCondenser = -HeatCondenser/(Heattransfercoefficient_KC*(DeltaT_Condenser)) #m2
CapitalCostCondenserB1 = 7296*(AreaCondenser**0.65)
                ## REBOILER  B1
Heattransfercoefficient_KR = 0.568 # kW/ K *m2 
T_outputReboiler  = Application.Tree.FindNode(BOTTOM_TEMP).Value + 273.15 #Kelvyn
Tvapor = 164 + 273.15 # Kelvyn Vapor of 100 Psi
HeatReboiler = Application.Tree.FindNode(REB_DUTY).Value*0.0041868    #cal/sec to kW
DeltaT_Reboiler = Tvapor - T_outputReboiler
AreaReboiler = HeatReboiler/(Heattransfercoefficient_KR *DeltaT_Reboiler )
CapitalCostReboilerB1 =  7296*(AreaReboiler**0.65)
TotalCapitalCost  = CapitalCostCondenserB1 +  CapitalCostColumnB1 + CapitalCostReboilerB1
TAC = (TotalCapitalCost/3)/1e6
print('TAC',TAC)

Composicion final de acetona calculado:  0.999805175
TAC 0.10145890276908356


In [10]:
Application.Quit(aspen_Path)